# スクレイピングの基礎

In [20]:
import requests
import pandas as pd
import time
from bs4 import BeautifulSoup

# とりあえずやってみる

In [21]:
time.sleep(1)
url = 'https://zenn.dev/topics/python'
res = requests.get(url)
soup = BeautifulSoup(res.text, 'html.parser')

## 参考
- https://zenn.dev/robots.txt
- [スクレイピング、クローリングする時の注意点](https://docs.pyq.jp/column/crawler.html)
- [最低限覚えておきたいCSSのセレクタ](https://morizyun.github.io/web/css-selector.html)
- [図解！Python BeautifulSoupの使い方を徹底解説！](https://ai-inter1.com/beautifulsoup_1/)

In [22]:
article_list = soup.select('[class^=ArticleList_content]')
sample = article_list[0]
# pretify()で整形
print(sample.prettify())

<div class="ArticleList_content__i6AQy">
 <a class="ArticleList_link__vf_6E" href="/irugo/articles/06a67373aa713a">
  <h2 class="ArticleList_title__P6X2G">
   Pythonのlistをもっと便利に +α 【大規模ソフトウェアを手探る】
  </h2>
 </a>
 <div class="ArticleList_user__FR8ks">
  <div class="ArticleList_avatar__oc2gR">
   <a href="/irugo">
    <img alt="いるんご" class="AvatarImage_plain__BCJNs" height="26" loading="lazy" referrerpolicy="no-referrer" src="https://res.cloudinary.com/zenn/image/fetch/s--HezPionX--/c_limit%2Cf_auto%2Cfl_progressive%2Cq_auto%2Cw_70/https://storage.googleapis.com/zenn-user-upload/avatar/26485c73a9.jpeg" width="26"/>
   </a>
  </div>
  <div class="ArticleList_userInfo__uTs5A">
   <div class="ArticleList_userName__GWXDx">
    <a href="/irugo">
     いるんご
    </a>
   </div>
   <div class="ArticleList_meta__E1zr4">
    <time class="ArticleList_date__L543S" datetime="2022-11-09T13:17:29+00:00">
     3日前
    </time>
    <span class="ArticleList_like__c4148">
     <svg aria-label="いいねされた数" height=

In [23]:
def get_article_info(article):
    title = article.select_one("h2").text
    author_name = article.select_one("[class^=ArticleList_userName]").text
    relative_link = article.select_one("[class^=ArticleList_link]").get("href")
    like = (
        article.select_one("[class^=ArticleList_like]").text
        if article.select_one("[class^=ArticleList_like]")
        else 0
    )
    return {"タイトル": title, "著者": author_name, "リンク": f'https://zenn.dev{relative_link}', "いいね": int(like)}

In [24]:
article_info_list =  []
for article in article_list:
    article_info_list.append(get_article_info(article))

df = pd.DataFrame(article_info_list)
# リンクをクリックできるようにする
df.style.format({'リンク': lambda x: f'<a href="{x}">{x}</a>'})

,タイトル,著者,リンク,いいね
0,Pythonのlistをもっと便利に +α 【大規模ソフトウェアを手探る】,いるんご,https://zenn.dev/irugo/articles/06a67373aa713a,36
1,エリア面積計算の多面的アプローチ,Luup Developers Blog,https://zenn.dev/luup/articles/data-oshima-20221105,24
2,AtCoderのとある問題でCode Golfに挑戦した記録 (Python3),sugyan,https://zenn.dev/sugyan/articles/5f45e4808ffc95,2
3,【python】PDFをスクレイピングして中身をCSVへ出力する（前編）,subaru-hello,https://zenn.dev/subaru_hello/articles/73253a46a76f7f,2
4,matplotlibの備忘録,canard0328,https://zenn.dev/canard0328/articles/memorandum-of-matplotlib,2
5,【Python】CSVファイルへの書き出しについて【備忘録】,SiganAI,https://zenn.dev/siganai/articles/20221112_python_csv_output,0
6,PyScript試してみた,Sawa,https://zenn.dev/sawa_tszm/articles/6dfdda21d5d702,6
7,macOS に bpy をインストールする,calcifer,https://zenn.dev/nek0n0mimi/articles/4f461c2e3ebc45,0
8,Python+selenumでchromeを動かすまで（Ubuntu）,プロトケラトプス,https://zenn.dev/honehone/articles/482f9a3b5ca481,2
9,Raspberry Pi 4 model Bで仮想環境をつくる,Tokoroteen,https://zenn.dev/tokoroteen/articles/c58b4a16269bad,1


## 関数化してページネーション対応

処理を分けて関数化することにより、使い回しが効く。

それだけでなくカプセル化としての役割もあることから、予期せぬエラーを防ぐことができる。

また、全体を見たとき、処理の流れがわかりやすくなる。

In [25]:
def get_soup(url):
    time.sleep(1)
    res = requests.get(url)
    soup = BeautifulSoup(res.text, 'html.parser')
    return soup

def get_articles(soup):
    article_list = soup.select('[class^=ArticleList_content]')
    return article_list

def get_article_info(article):
    title = article.select_one("h2").text
    author_name = article.select_one("[class^=ArticleList_userName]").text
    relative_link = article.select_one("[class^=ArticleList_link]").get("href")
    like = (
        article.select_one("[class^=ArticleList_like]").text
        if article.select_one("[class^=ArticleList_like]")
        else 0
    )
    return {"タイトル": title, "著者": author_name, "リンク": f'https://zenn.dev{relative_link}', "いいね": int(like)}

def get_article_info_df(url):
    soup = get_soup(url)
    article_list = get_articles(soup)
    article_info_list =  []
    for article in article_list:
        article_info_list.append(get_article_info(article))
    df = pd.DataFrame(article_info_list)
    return df

In [41]:
def get_zenn_article_df_by_topic(max_page, topic="python", verbose=False):
    urls = [f"https://zenn.dev/topics/{topic}?page={i}" for i in range(1, max_page + 1)]

    df_list = []
    for url in urls:
        try:
            df = get_article_info_df(url)
            df_list.append(df)
            if verbose:
                print(f"success: {url}")
        except:
            print(f"error!: {url}の取得に失敗しました")
    df = pd.concat(df_list)
    return df

In [42]:
df = get_zenn_article_df_by_topic(3)

In [46]:
df.sort_values('いいね', ascending=False).head(10).style.format({'リンク': lambda x: f'<a href="{x}">{x}</a>'})

,タイトル,著者,リンク,いいね
45,君には今から3時間で機械学習Webアプリを作ってもらうよ,alivelimb,https://zenn.dev/alivelimb/articles/20220528-streamlit-ml-app,427
46,Python互換の静的型付け言語「Erg」,shiba,https://zenn.dev/mtshiba/articles/a38c9fcd9646d4,339
26,話題のStable Diffusionがオープンソース化されたのでローカルで動かしてみる,koyoarai_,https://zenn.dev/koyoarai_/articles/02f3ed864c6127bb2049,339
22,Pythonでの開発・CI/CDの私的ベストプラクティス2022,dajiaji,https://zenn.dev/dajiaji/articles/e29005502a9e78,170
31,中級者へのModern Python,ganyariya,https://zenn.dev/ganariya/articles/intermediate-python,170
40,2021年Python開発リンター導入のベストプラクティス,Yusuke,https://zenn.dev/yhay81/articles/yhay81-202102-pythonlint,145
47,データ収集から機械学習まで全て行って競馬の予測をしてみた,kami,https://zenn.dev/kami/articles/66e400c8a43cd08a5d7f,142
0,データ収集から機械学習まで全て行って競馬の予測をしてみた,kami,https://zenn.dev/kami/articles/66e400c8a43cd08a5d7f,142
39,私が考えるLambda開発環境のベストプラクティス,Dai@FastLabel,https://zenn.dev/faycute/articles/9024115c4241b8,122
16,Pythonで自分だけのクソライブラリを作る方法,karaage0703,https://zenn.dev/karaage0703/articles/db8c663640c68b,122
